# Scenario

How to use LangChain with Azure OpenAI focusing on the new multiple tools feature and optionally disabling the parallel tools calling.

## The imports

In [1]:
from langchain_openai import AzureChatOpenAI


## Define our function calls aka "tools"

In [2]:
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

tools = [add, multiply]
tools

[<function __main__.add(a: int, b: int) -> int>,
 <function __main__.multiply(a: int, b: int) -> int>]

## Disabling parallel tool calling just before invoking the tools call

### Initialize the client. Notice no special parameters at this time.

In [17]:
llm = AzureChatOpenAI(
    azure_endpoint="YOUR_AZURE_ENDPOINT",
    deployment_name="gpt-4-0",
    model_name="gpt-4o",
    model_version="2024-05-13",
    api_version="2024-08-01-preview",
    api_key="YOUR_AZURE_API_KEY",
    max_tokens=50
)

### Start with default tools calling.
Notice that both 'add' and 'multiply' operations (tools) are invoked.

In [18]:
query = "What is 3 * 12? Also, what is 11 + 49?"
llm_with_tools = llm.bind_tools(tools)
llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_GHsBGHenjrA90n0xyP0FSGMy',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_0Za7bxcAqS2Trh4Wd7FXZQk0',
  'type': 'tool_call'}]

### Now set parallel tool calling = false just before invoking.
Notice that only the first operations (tool) is invoked.

In [19]:
query = "What is 3 * 12? Also, what is 11 + 49?"
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)
llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_5Q9qZZvDDzWu5zoJ6JBjW3LJ',
  'type': 'tool_call'}]

## Disabling parallel tool calling during client initialization

In [20]:
llm = AzureChatOpenAI(
    azure_endpoint="YOUR_AZURE_ENDPOINT",
    deployment_name="gpt-4-0",
    model_name="gpt-4o",
    model_version="2024-05-13",
    api_version="2024-08-01-preview",
    api_key="YOUR_AZURE_API_KEY",
    max_tokens=50,
    model_kwargs={
       "parallel_tool_calls" : False
    }
)

### Invoke again the same tools with the same prompt
Notice that the first MULTIPLY operation (tool call) is logged.

In [21]:
llm_with_tools = llm.bind_tools(tools)
query = "What is 3 * 12? Also, what is 11 + 49?"
llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_D6oQHIqmvGPgX24gQJaVQV6G',
  'type': 'tool_call'}]

### Invoke again but switch the operations in the prompt.
Notice that this time, the ADD operation (tool call) is logged.

In [22]:
llm_with_tools = llm.bind_tools(tools)
query = "What is 11 + 49? Also, what is 3 * 12?"
llm_with_tools.invoke(query).tool_calls

[{'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_HxA2sLq9tBoBuXAfMKov4r0O',
  'type': 'tool_call'}]